In [1]:
# Copyright (c) Meta Platforms, Inc. and affiliates.
# All rights reserved.

# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.
# --------------------------------------------------------
# References:
# DeiT: https://github.com/facebookresearch/deit
# BEiT: https://github.com/microsoft/unilm/tree/master/beit
# --------------------------------------------------------

import timm.optim.optim_factory as optim_factory
import argparse
import datetime
import json
import numpy as np
import os
import time
from pathlib import Path

import torch
import torch.backends.cudnn as cudnn
from torch.utils.tensorboard import SummaryWriter

import timm

# assert timm.__version__ == "0.3.2" # version check
from timm.models.layers import trunc_normal_
from timm.data.mixup import Mixup
from timm.loss import LabelSmoothingCrossEntropy, SoftTargetCrossEntropy

import util.lr_decay as lrd
import util.misc as misc
from util.datasets import build_dataset
from util.pos_embed import interpolate_pos_embed
from util.misc import NativeScalerWithGradNormCount as NativeScaler

import models_vit

from engine_finetune import train_one_epoch, evaluate


def get_args_parser():
    parser = argparse.ArgumentParser('MAE fine-tuning for image segmentation', add_help=False)
    parser.add_argument('--batch_size', default=64, type=int,
                        help='Batch size per GPU (effective batch size is batch_size * accum_iter * # gpus')
    parser.add_argument('--epochs', default=300, type=int)
    parser.add_argument('--accum_iter', default=1, type=int,
                        help='Accumulate gradient iterations (for increasing the effective batch size under memory constraints)')

    # Model parameters
    parser.add_argument('--model', default='mae_vit_base_patch16', type=str, metavar='MODEL',
                        help='Name of model to train')

    parser.add_argument('--input_size', default=224, type=int,
                        help='images input size')

    parser.add_argument('--drop_path', type=float, default=0.1, metavar='PCT',
                        help='Drop path rate (default: 0.1)')

    # Optimizer parameters
    parser.add_argument('--clip_grad', type=float, default=None, metavar='NORM',
                        help='Clip gradient norm (default: None, no clipping)')
    parser.add_argument('--weight_decay', type=float, default=0.0005,
                        help='weight decay (default: 0.05)')

    parser.add_argument('--lr', type=float, default=1e-4, metavar='LR',
                        help='learning rate (absolute lr)')
    parser.add_argument('--blr', type=float, default=1e-4, metavar='LR',
                        help='base learning rate: absolute_lr = base_lr * total_batch_size / 256')
    parser.add_argument('--layer_decay', type=float, default=0.5,
                        help='layer-wise lr decay from ELECTRA/BEiT')

    parser.add_argument('--min_lr', type=float, default=1e-6, metavar='LR',
                        help='lower lr bound for cyclic schedulers that hit 0')

    parser.add_argument('--warmup_epochs', type=int, default=5, metavar='N',
                        help='epochs to warmup LR')

    # Augmentation parameters
    parser.add_argument('--color_jitter', type=float, default=None, metavar='PCT',
                        help='Color jitter factor (enabled only when not using Auto/RandAug)')
    parser.add_argument('--aa', type=str, default='rand-m9-mstd0.5-inc1', metavar='NAME',
                        help='Use AutoAugment policy. "v0" or "original". " + "(default: rand-m9-mstd0.5-inc1)'),
    parser.add_argument('--smoothing', type=float, default=0.1,
                        help='Label smoothing (default: 0.1)')

    # * Random Erase params
    parser.add_argument('--reprob', type=float, default=0.25, metavar='PCT',
                        help='Random erase prob (default: 0.25)')
    parser.add_argument('--remode', type=str, default='pixel',
                        help='Random erase mode (default: "pixel")')
    parser.add_argument('--recount', type=int, default=1,
                        help='Random erase count (default: 1)')
    parser.add_argument('--resplit', action='store_true', default=False,
                        help='Do not random erase first (clean) augmentation split')

    # * Mixup params
    parser.add_argument('--mixup', type=float, default=0,
                        help='mixup alpha, mixup enabled if > 0.')
    parser.add_argument('--cutmix', type=float, default=0,
                        help='cutmix alpha, cutmix enabled if > 0.')
    parser.add_argument('--cutmix_minmax', type=float, nargs='+', default=None,
                        help='cutmix min/max ratio, overrides alpha and enables cutmix if set (default: None)')
    parser.add_argument('--mixup_prob', type=float, default=1.0,
                        help='Probability of performing mixup or cutmix when either/both is enabled')
    parser.add_argument('--mixup_switch_prob', type=float, default=0.5,
                        help='Probability of switching to cutmix when both mixup and cutmix enabled')
    parser.add_argument('--mixup_mode', type=str, default='batch',
                        help='How to apply mixup/cutmix params. Per "batch", "pair", or "elem"')

    # * Finetuning params
    parser.add_argument('--finetune', default='/data/zhanghao/skull_project/mae-main/pretrain_output_dir/checkpoint-399.pth',
                        help='finetune from checkpoint')
    parser.add_argument('--global_pool', action='store_true')
    parser.set_defaults(global_pool=True)
    parser.add_argument('--cls_token', action='store_false', dest='global_pool',
                        help='Use class token instead of global pool for classification')

    # Dataset parameters
    parser.add_argument('--data_path', default='/datasets01/imagenet_full_size/061417/', type=str,
                        help='dataset path')
    parser.add_argument('--nb_classes', default=1000, type=int,
                        help='number of the classification types')

    parser.add_argument('--output_dir', default='./mae-main/base_line/mae2d_oasis_020data',
                        help='path where to save, empty for no saving')
    parser.add_argument('--log_dir', default='./mae-main/base_line/mae2d_oasis_020data',
                        help='path where to tensorboard log')
    parser.add_argument('--device', default='cuda:2',
                        help='device to use for training / testing')
    parser.add_argument('--seed', default=0, type=int)
    parser.add_argument('--resume', default='',
                        help='resume from checkpoint')

    parser.add_argument('--start_epoch', default=0, type=int, metavar='N',
                        help='start epoch')
    parser.add_argument('--eval', action='store_true',
                        help='Perform evaluation only')
    parser.add_argument('--dist_eval', action='store_true', default=False,
                        help='Enabling distributed evaluation (recommended during training for faster monitor')
    parser.add_argument('--num_workers', default=10, type=int)
    parser.add_argument('--pin_mem', action='store_true',
                        help='Pin CPU memory in DataLoader for more efficient (sometimes) transfer to GPU.')
    parser.add_argument('--no_pin_mem', action='store_false', dest='pin_mem')
    parser.set_defaults(pin_mem=True)

    # distributed training parameters
    parser.add_argument('--world_size', default=1, type=int,
                        help='number of distributed processes')
    parser.add_argument('--local_rank', default=-1, type=int)
    parser.add_argument('--dist_on_itp', action='store_true')
    parser.add_argument('--dist_url', default='env://',
                        help='url used to set up distributed training')

    return parser

In [2]:
from util.oasisdataset import *
train_path, valid_path, test_path = getPathList()
train_loader = getDataloader020(
    train_path, B1=96)
_,valid_loader,test_loader = getDataloader(
    train_path,valid_path,test_path
)
len(train_loader),len(valid_loader),len(test_loader)
#(1,1,160,224,224)

(114, 6720, 6880)

# 不冻结decoder权重的finetune

In [4]:
import my_seg_vit
import util.lr_sched as lr_sched
import math
import sys
from my_metric import dice_coef_metric,jaccard_coef_metric
def main(args):   
     
    device = torch.device(args.device)

    os.makedirs(args.log_dir, exist_ok=True)
    log_writer = SummaryWriter(log_dir=args.log_dir)

    # fix the seed for reproducibility
    seed = args.seed + misc.get_rank()
    torch.manual_seed(seed)
    np.random.seed(seed) 

    cudnn.benchmark = True   

    model = my_seg_vit.__dict__[args.model](lossforpatch = False)

    #load dict
    checkpoint = torch.load(args.finetune,map_location='cpu') #一大堆参数(小数)
    print("Load pre-trained checkpoint from: %s" % args.finetune)
    checkpoint_model = checkpoint['model']
    
    state_dict = model.state_dict()
    # print('state_dict = ',state_dict)

    for k in ['head.weight', 'head.bias']:
        if k in checkpoint_model and checkpoint_model[k].shape != state_dict[k].shape:
            print(f"Removing key {k} from pretrained checkpoint")
            del checkpoint_model[k]
    
    # interpolate position embedding
    interpolate_pos_embed(model, checkpoint_model)

    # load pre-trained model
    msg = model.load_state_dict(checkpoint_model, strict=False)
    print(msg)

    model.to(device)

    model_without_ddp = model


    n_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)

    print("Model = %s" % str(model_without_ddp))
    print('number of params (M): %.2f' % (n_parameters / 1.e6))

    eff_batch_size = args.batch_size * args.accum_iter * misc.get_world_size()


    print("actual lr: %.2e" % args.lr)

    print("accumulate grad iterations: %d" % args.accum_iter)
    print("effective batch size: %d" % eff_batch_size)

    #加了layer_decay
    param_groups = lrd.param_groups_lrd(model_without_ddp, args.weight_decay,
        layer_decay=args.layer_decay
    )
    # param_groups = optim_factory.param_groups_weight_decay(
    #     model_without_ddp, args.weight_decay)
 
    optimizer = torch.optim.AdamW(param_groups, lr=args.lr)
    print("查看optimizer = ",optimizer)
    loss_scaler = NativeScaler()    

    misc.load_model(args=args, model_without_ddp=model_without_ddp, optimizer=optimizer, loss_scaler=loss_scaler)

    print(f"Start training for {args.epochs} epochs")
    dice_list = []
    start_time = time.time()

    for epoch in range(args.start_epoch, args.epochs):
        #----------------------------训练---------------------------------------------
        model.train()
        metric_logger = misc.MetricLogger(delimiter="  ")
        metric_logger.add_meter('lr', misc.SmoothedValue(window_size=1, fmt='{value:.6f}'))
        accum_iter = args.accum_iter
        optimizer.zero_grad()#优化器清零
        if log_writer is not None:
            print('log_dir: {}'.format(log_writer.log_dir))

        for data_iter_step,data in enumerate(train_loader):
            if data_iter_step % accum_iter == 0:
                lr_sched.adjust_learning_rate(optimizer, data_iter_step / len(train_loader) + epoch, args)
            samples = data["image"]  # 我自己的数据是data["image"]表示图片
            labeles = data["mask"]
            # print("mask.shape = ",labeles.shape)
            # print("samples.shape = ",samples.shape)
            # print("mask.type = ",labeles.dtype)
            samples = samples.to(device, non_blocking=True)
            labeles = labeles.to(device, non_blocking=True)

            with torch.cuda.amp.autocast():
                loss, _,  = model(samples, labeles)

            loss_value = loss.item()

            if not math.isfinite(loss_value):
                print("Loss is {}, stopping training".format(loss_value))
                sys.exit(1)

            loss /= accum_iter
            loss_scaler(loss, optimizer, parameters=model.parameters(),update_grad=(data_iter_step + 1) % accum_iter == 0)

            if (data_iter_step + 1) % accum_iter == 0:
                optimizer.zero_grad()

            torch.cuda.synchronize()

            metric_logger.update(loss=loss_value)

            lr = optimizer.param_groups[0]["lr"]
            metric_logger.update(lr=lr)

            loss_value_reduce = misc.all_reduce_mean(loss_value)

            if log_writer is not None :
                """ We use epoch_1000x as the x-axis in tensorboard.
                This calibrates different curves when batch size changes.
                """
                epoch_1000x = int(
                    (data_iter_step / len(train_loader) + epoch) * 1000)
                log_writer.add_scalar('train_loss', loss_value_reduce, epoch_1000x)
                log_writer.add_scalar('lr', lr, epoch_1000x)

        # gather the stats from all processes
        metric_logger.synchronize_between_processes()
        print("Averaged stats:", metric_logger)
        train_stats = {k: meter.global_avg for k, meter in metric_logger.meters.items()}

        #----------------------------验证-----------------------------------------------
        #每4个epoch进行一次验证
        if(epoch%4==0):
            with torch.no_grad(): #表示在验证的时候不需要梯度计算
                model.eval()
                dice_score = 0.
                iou_score = 0.
                for data in valid_loader:
                    valid_image = data["image"]
                    valid_target = data["mask"]

                    valid_image = valid_image.to(device, non_blocking=True)
                    valid_target = valid_target.to(device, non_blocking=True)

                    _,out = model(valid_image, valid_target) #[N,196,256]
                    out = model.unpatchify(out) #[N,1,224,224]
                    dice_score += dice_coef_metric(out.cpu(), valid_target.cpu())
                    iou_score += jaccard_coef_metric(out.cpu(), valid_target.cpu())
                
                dice_score /= len(valid_loader)
                iou_score /= len(valid_loader)
                print("epoch=", epoch,
                    "dice_score=", dice_score,
                    "iou_score=", iou_score)

                print("------------------------------------------------------------------")
                log_writer.add_scalar(tag="dice_scalar", scalar_value=dice_score, global_step=epoch)
                log_writer.add_scalar(tag="iou_scalar", scalar_value=iou_score, global_step=epoch)
                #原作者的save方法,封装更多内容 
                misc.save_model(args=args, model=model, model_without_ddp=model, optimizer=optimizer,
                loss_scaler=loss_scaler, epoch=epoch)
                #torch官方save方法
                torch.save(model.state_dict(), '/data/zhanghao/skull_project/mae-main/base_line/mae2d_oasis_020data/' +
                                    str(epoch) + 'ViT' + str(dice_score) + '.pth')
                

            #早停
            if epoch > 20:
                min = 999
                for i in range(int(epoch/4 - 5) , int(epoch/4 - 1) ):
                    if dice_list[i] < min:
                        min = dice_list[i]
                if dice_score < min:
                    lr /= 10
                    print(lr)
                    if lr < 1e-6:
                        break

            dice_list.append(dice_score)#依次放入epoch为0 4 8 12 16 20 24....

        log_stats = {**{f'train_{k}': v for k, v in train_stats.items()},
                     'epoch': epoch, }

        if args.output_dir and misc.is_main_process():
            if log_writer is not None:
                log_writer.flush()
            with open(os.path.join(args.output_dir, "log.txt"), mode="a", encoding="utf-8") as f:
                f.write(json.dumps(log_stats) + "\n")

    total_time = time.time() - start_time
    total_time_str = str(datetime.timedelta(seconds=int(total_time)))
    print('Training time {}'.format(total_time_str))

            

In [5]:
args = get_args_parser()
args = args.parse_args(args=[])
if args.output_dir:
    Path(args.output_dir).mkdir(parents=True, exist_ok=True)
main(args)        
    

Load pre-trained checkpoint from: /data/zhanghao/skull_project/mae-main/pretrain_output_dir/checkpoint-399.pth
_IncompatibleKeys(missing_keys=[], unexpected_keys=['mask_token'])
Model = SegViT2D(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(1, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (blocks): ModuleList(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU()
        (drop1): Dropout(p=0.0, inplace=False)
        (fc2): Linea

# 测试

In [35]:
from util.oasisdataset3d import *
import my_seg_vit
import util.lr_sched as lr_sched
import math
import sys
from my_metric import dice_coef_metric,jaccard_coef_metric
train_path, valid_path, test_path = getPathList()

train_loader,valid_loader,test_loader = getDataloader(
    train_path,valid_path,test_path
)
len(train_loader),len(valid_loader),len(test_loader)
#(1,1,160,224,224)

(340, 42, 43)

In [38]:

model_str = 'mae_vit_base_patch16'
model_path = '/data/zhanghao/skull_project/mae-main/base_line/mae2d_oasis_005data/84ViT0.9458257578022151.pth'
model = my_seg_vit.__dict__[model_str](lossforpatch = False)
model.load_state_dict(torch.load(model_path))
model = model.cuda(1)

In [39]:
with torch.no_grad(): #表示在验证的时候不需要梯度计算
    model.eval()
    dice_score = 0.
    iou_score = 0.
    for data in test_loader:
        test_image = data["image"]
        test_target = data["mask"]

        test_image = test_image.squeeze(0)


        test_image = test_image.permute(1,0,2,3)


        test_image = test_image.to('cuda:1', non_blocking=True)
        test_target = test_target.to('cuda:1', non_blocking=True)
        
        # assert test_image.shape == (160,1,224,224)
        # print(test_image.shape)

        _,out = model(test_image, test_image) #[N,196,256]
        out = model.unpatchify(out) #[N,1,224,224]

        out = out.permute(1,0,2,3)
        out = out.unsqueeze(0)
        

        dice_score += dice_coef_metric(out.cpu(), test_target.cpu())
        iou_score += jaccard_coef_metric(out.cpu(), test_target.cpu())
        print(dice_coef_metric(out.cpu(), test_target.cpu()))
    
    dice_score /= len(test_loader)
    iou_score /= len(test_loader)
    print(
        "dice_score=", dice_score,
        "iou_score=", iou_score)

0.97259057
0.97453684
0.9661456
0.974032
0.9455001
0.9483417
0.96562725
0.9770106
0.9705029
0.9660105
0.96337664
0.96864563
0.9370107
0.9658173
0.9630881
0.9753735
0.9706026
0.97138125
0.96688896
0.96548617
0.9508345
0.96207744
0.96851665
0.97413224
0.9342987
0.9458862
0.9657937
0.9725448
0.9438343
0.9694775
0.95929694
0.9727952
0.9484608
0.974316
0.93850905
0.97650695
0.9703403
0.9628539
0.96314144
0.97340715
0.9515461
0.97162366
0.9701765
dice_score= 0.9634497415187747 iou_score= 0.9297138521837633


: 